In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [51]:
#renumbered as conjecture 9

import pickle
rsfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

mp2=2
data=[]
p=2
t_set=[]
dk=2
#dkd=dk.digits(p)
for h in [0..len(s)-1]:
    k=s[h][0]
    if digitsum(p,k)==dk:
        poly=D(s[h][1])
        for m in [3..1003]:
                if mp2==mod(m,p^2): # m = 2 (mod 4)
                    polym=evaluate(poly,m)
                    o=order(p,polym)
                    a=order(p,m) # a = 1 bc m = 2 (mod 4)
                    t=(a+6)*k+2-o 
                    t_set=t_set+[t]
                    tupl=(k,m,t)
                    print(tupl)
print()
print("set union of t_set:")
print(uniteList(t_set))

(3, 6, 0)
(3, 10, 0)
(3, 14, 0)
(3, 18, 0)
(3, 22, 0)
(3, 26, 0)
(3, 30, 0)
(3, 34, 0)
(3, 38, 0)
(3, 42, 0)
(3, 46, 0)
(3, 50, 0)
(3, 54, 0)
(3, 58, 0)
(3, 62, 0)
(3, 66, 0)
(3, 70, 0)
(3, 74, 0)
(3, 78, 0)
(3, 82, 0)
(3, 86, 0)
(3, 90, 0)
(3, 94, 0)
(3, 98, 0)
(3, 102, 0)
(3, 106, 0)
(3, 110, 0)
(3, 114, 0)
(3, 118, 0)
(3, 122, 0)
(3, 126, 0)
(3, 130, 0)
(3, 134, 0)
(3, 138, 0)
(3, 142, 0)
(3, 146, 0)
(3, 150, 0)
(3, 154, 0)
(3, 158, 0)
(3, 162, 0)
(3, 166, 0)
(3, 170, 0)
(3, 174, 0)
(3, 178, 0)
(3, 182, 0)
(3, 186, 0)
(3, 190, 0)
(3, 194, 0)
(3, 198, 0)
(3, 202, 0)
(3, 206, 0)
(3, 210, 0)
(3, 214, 0)
(3, 218, 0)
(3, 222, 0)
(3, 226, 0)
(3, 230, 0)
(3, 234, 0)
(3, 238, 0)
(3, 242, 0)
(3, 246, 0)
(3, 250, 0)
(3, 254, 0)
(3, 258, 0)
(3, 262, 0)
(3, 266, 0)
(3, 270, 0)
(3, 274, 0)
(3, 278, 0)
(3, 282, 0)
(3, 286, 0)
(3, 290, 0)
(3, 294, 0)
(3, 298, 0)
(3, 302, 0)
(3, 306, 0)
(3, 310, 0)
(3, 314, 0)
(3, 318, 0)
(3, 322, 0)
(3, 326, 0)
(3, 330, 0)
(3, 334, 0)
(3, 338, 0)
(3, 342, 0)
(3, 3

(9, 550, 0)
(9, 554, 0)
(9, 558, 0)
(9, 562, 0)
(9, 566, 0)
(9, 570, 0)
(9, 574, 0)
(9, 578, 0)
(9, 582, 0)
(9, 586, 0)
(9, 590, 0)
(9, 594, 0)
(9, 598, 0)
(9, 602, 0)
(9, 606, 0)
(9, 610, 0)
(9, 614, 0)
(9, 618, 0)
(9, 622, 0)
(9, 626, 0)
(9, 630, 0)
(9, 634, 0)
(9, 638, 0)
(9, 642, 0)
(9, 646, 0)
(9, 650, 0)
(9, 654, 0)
(9, 658, 0)
(9, 662, 0)
(9, 666, 0)
(9, 670, 0)
(9, 674, 0)
(9, 678, 0)
(9, 682, 0)
(9, 686, 0)
(9, 690, 0)
(9, 694, 0)
(9, 698, 0)
(9, 702, 0)
(9, 706, 0)
(9, 710, 0)
(9, 714, 0)
(9, 718, 0)
(9, 722, 0)
(9, 726, 0)
(9, 730, 0)
(9, 734, 0)
(9, 738, 0)
(9, 742, 0)
(9, 746, 0)
(9, 750, 0)
(9, 754, 0)
(9, 758, 0)
(9, 762, 0)
(9, 766, 0)
(9, 770, 0)
(9, 774, 0)
(9, 778, 0)
(9, 782, 0)
(9, 786, 0)
(9, 790, 0)
(9, 794, 0)
(9, 798, 0)
(9, 802, 0)
(9, 806, 0)
(9, 810, 0)
(9, 814, 0)
(9, 818, 0)
(9, 822, 0)
(9, 826, 0)
(9, 830, 0)
(9, 834, 0)
(9, 838, 0)
(9, 842, 0)
(9, 846, 0)
(9, 850, 0)
(9, 854, 0)
(9, 858, 0)
(9, 862, 0)
(9, 866, 0)
(9, 870, 0)
(9, 874, 0)
(9, 878, 0)
(9, 

(17, 206, 0)
(17, 210, 0)
(17, 214, 0)
(17, 218, 0)
(17, 222, 0)
(17, 226, 0)
(17, 230, 0)
(17, 234, 0)
(17, 238, 0)
(17, 242, 0)
(17, 246, 0)
(17, 250, 0)
(17, 254, 0)
(17, 258, 0)
(17, 262, 0)
(17, 266, 0)
(17, 270, 0)
(17, 274, 0)
(17, 278, 0)
(17, 282, 0)
(17, 286, 0)
(17, 290, 0)
(17, 294, 0)
(17, 298, 0)
(17, 302, 0)
(17, 306, 0)
(17, 310, 0)
(17, 314, 0)
(17, 318, 0)
(17, 322, 0)
(17, 326, 0)
(17, 330, 0)
(17, 334, 0)
(17, 338, 0)
(17, 342, 0)
(17, 346, 0)
(17, 350, 0)
(17, 354, 0)
(17, 358, 0)
(17, 362, 0)
(17, 366, 0)
(17, 370, 0)
(17, 374, 0)
(17, 378, 0)
(17, 382, 0)
(17, 386, 0)
(17, 390, 0)
(17, 394, 0)
(17, 398, 0)
(17, 402, 0)
(17, 406, 0)
(17, 410, 0)
(17, 414, 0)
(17, 418, 0)
(17, 422, 0)
(17, 426, 0)
(17, 430, 0)
(17, 434, 0)
(17, 438, 0)
(17, 442, 0)
(17, 446, 0)
(17, 450, 0)
(17, 454, 0)
(17, 458, 0)
(17, 462, 0)
(17, 466, 0)
(17, 470, 0)
(17, 474, 0)
(17, 478, 0)
(17, 482, 0)
(17, 486, 0)
(17, 490, 0)
(17, 494, 0)
(17, 498, 0)
(17, 502, 0)
(17, 506, 0)
(17, 510, 0)

(20, 998, 0)
(20, 1002, 0)
(24, 6, 0)
(24, 10, 0)
(24, 14, 0)
(24, 18, 0)
(24, 22, 0)
(24, 26, 0)
(24, 30, 0)
(24, 34, 0)
(24, 38, 0)
(24, 42, 0)
(24, 46, 0)
(24, 50, 0)
(24, 54, 0)
(24, 58, 0)
(24, 62, 0)
(24, 66, 0)
(24, 70, 0)
(24, 74, 0)
(24, 78, 0)
(24, 82, 0)
(24, 86, 0)
(24, 90, 0)
(24, 94, 0)
(24, 98, 0)
(24, 102, 0)
(24, 106, 0)
(24, 110, 0)
(24, 114, 0)
(24, 118, 0)
(24, 122, 0)
(24, 126, 0)
(24, 130, 0)
(24, 134, 0)
(24, 138, 0)
(24, 142, 0)
(24, 146, 0)
(24, 150, 0)
(24, 154, 0)
(24, 158, 0)
(24, 162, 0)
(24, 166, 0)
(24, 170, 0)
(24, 174, 0)
(24, 178, 0)
(24, 182, 0)
(24, 186, 0)
(24, 190, 0)
(24, 194, 0)
(24, 198, 0)
(24, 202, 0)
(24, 206, 0)
(24, 210, 0)
(24, 214, 0)
(24, 218, 0)
(24, 222, 0)
(24, 226, 0)
(24, 230, 0)
(24, 234, 0)
(24, 238, 0)
(24, 242, 0)
(24, 246, 0)
(24, 250, 0)
(24, 254, 0)
(24, 258, 0)
(24, 262, 0)
(24, 266, 0)
(24, 270, 0)
(24, 274, 0)
(24, 278, 0)
(24, 282, 0)
(24, 286, 0)
(24, 290, 0)
(24, 294, 0)
(24, 298, 0)
(24, 302, 0)
(24, 306, 0)
(24, 310, 

(34, 550, 0)
(34, 554, 0)
(34, 558, 0)
(34, 562, 0)
(34, 566, 0)
(34, 570, 0)
(34, 574, 0)
(34, 578, 0)
(34, 582, 0)
(34, 586, 0)
(34, 590, 0)
(34, 594, 0)
(34, 598, 0)
(34, 602, 0)
(34, 606, 0)
(34, 610, 0)
(34, 614, 0)
(34, 618, 0)
(34, 622, 0)
(34, 626, 0)
(34, 630, 0)
(34, 634, 0)
(34, 638, 0)
(34, 642, 0)
(34, 646, 0)
(34, 650, 0)
(34, 654, 0)
(34, 658, 0)
(34, 662, 0)
(34, 666, 0)
(34, 670, 0)
(34, 674, 0)
(34, 678, 0)
(34, 682, 0)
(34, 686, 0)
(34, 690, 0)
(34, 694, 0)
(34, 698, 0)
(34, 702, 0)
(34, 706, 0)
(34, 710, 0)
(34, 714, 0)
(34, 718, 0)
(34, 722, 0)
(34, 726, 0)
(34, 730, 0)
(34, 734, 0)
(34, 738, 0)
(34, 742, 0)
(34, 746, 0)
(34, 750, 0)
(34, 754, 0)
(34, 758, 0)
(34, 762, 0)
(34, 766, 0)
(34, 770, 0)
(34, 774, 0)
(34, 778, 0)
(34, 782, 0)
(34, 786, 0)
(34, 790, 0)
(34, 794, 0)
(34, 798, 0)
(34, 802, 0)
(34, 806, 0)
(34, 810, 0)
(34, 814, 0)
(34, 818, 0)
(34, 822, 0)
(34, 826, 0)
(34, 830, 0)
(34, 834, 0)
(34, 838, 0)
(34, 842, 0)
(34, 846, 0)
(34, 850, 0)
(34, 854, 0)

(48, 106, 0)
(48, 110, 0)
(48, 114, 0)
(48, 118, 0)
(48, 122, 0)
(48, 126, 0)
(48, 130, 0)
(48, 134, 0)
(48, 138, 0)
(48, 142, 0)
(48, 146, 0)
(48, 150, 0)
(48, 154, 0)
(48, 158, 0)
(48, 162, 0)
(48, 166, 0)
(48, 170, 0)
(48, 174, 0)
(48, 178, 0)
(48, 182, 0)
(48, 186, 0)
(48, 190, 0)
(48, 194, 0)
(48, 198, 0)
(48, 202, 0)
(48, 206, 0)
(48, 210, 0)
(48, 214, 0)
(48, 218, 0)
(48, 222, 0)
(48, 226, 0)
(48, 230, 0)
(48, 234, 0)
(48, 238, 0)
(48, 242, 0)
(48, 246, 0)
(48, 250, 0)
(48, 254, 0)
(48, 258, 0)
(48, 262, 0)
(48, 266, 0)
(48, 270, 0)
(48, 274, 0)
(48, 278, 0)
(48, 282, 0)
(48, 286, 0)
(48, 290, 0)
(48, 294, 0)
(48, 298, 0)
(48, 302, 0)
(48, 306, 0)
(48, 310, 0)
(48, 314, 0)
(48, 318, 0)
(48, 322, 0)
(48, 326, 0)
(48, 330, 0)
(48, 334, 0)
(48, 338, 0)
(48, 342, 0)
(48, 346, 0)
(48, 350, 0)
(48, 354, 0)
(48, 358, 0)
(48, 362, 0)
(48, 366, 0)
(48, 370, 0)
(48, 374, 0)
(48, 378, 0)
(48, 382, 0)
(48, 386, 0)
(48, 390, 0)
(48, 394, 0)
(48, 398, 0)
(48, 402, 0)
(48, 406, 0)
(48, 410, 0)

(66, 690, 0)
(66, 694, 0)
(66, 698, 0)
(66, 702, 0)
(66, 706, 0)
(66, 710, 0)
(66, 714, 0)
(66, 718, 0)
(66, 722, 0)
(66, 726, 0)
(66, 730, 0)
(66, 734, 0)
(66, 738, 0)
(66, 742, 0)
(66, 746, 0)
(66, 750, 0)
(66, 754, 0)
(66, 758, 0)
(66, 762, 0)
(66, 766, 0)
(66, 770, 0)
(66, 774, 0)
(66, 778, 0)
(66, 782, 0)
(66, 786, 0)
(66, 790, 0)
(66, 794, 0)
(66, 798, 0)
(66, 802, 0)
(66, 806, 0)
(66, 810, 0)
(66, 814, 0)
(66, 818, 0)
(66, 822, 0)
(66, 826, 0)
(66, 830, 0)
(66, 834, 0)
(66, 838, 0)
(66, 842, 0)
(66, 846, 0)
(66, 850, 0)
(66, 854, 0)
(66, 858, 0)
(66, 862, 0)
(66, 866, 0)
(66, 870, 0)
(66, 874, 0)
(66, 878, 0)
(66, 882, 0)
(66, 886, 0)
(66, 890, 0)
(66, 894, 0)
(66, 898, 0)
(66, 902, 0)
(66, 906, 0)
(66, 910, 0)
(66, 914, 0)
(66, 918, 0)
(66, 922, 0)
(66, 926, 0)
(66, 930, 0)
(66, 934, 0)
(66, 938, 0)
(66, 942, 0)
(66, 946, 0)
(66, 950, 0)
(66, 954, 0)
(66, 958, 0)
(66, 962, 0)
(66, 966, 0)
(66, 970, 0)
(66, 974, 0)
(66, 978, 0)
(66, 982, 0)
(66, 986, 0)
(66, 990, 0)
(66, 994, 0)

(80, 294, 0)
(80, 298, 0)
(80, 302, 0)
(80, 306, 0)
(80, 310, 0)
(80, 314, 0)
(80, 318, 0)
(80, 322, 0)
(80, 326, 0)
(80, 330, 0)
(80, 334, 0)
(80, 338, 0)
(80, 342, 0)
(80, 346, 0)
(80, 350, 0)
(80, 354, 0)
(80, 358, 0)
(80, 362, 0)
(80, 366, 0)
(80, 370, 0)
(80, 374, 0)
(80, 378, 0)
(80, 382, 0)
(80, 386, 0)
(80, 390, 0)
(80, 394, 0)
(80, 398, 0)
(80, 402, 0)
(80, 406, 0)
(80, 410, 0)
(80, 414, 0)
(80, 418, 0)
(80, 422, 0)
(80, 426, 0)
(80, 430, 0)
(80, 434, 0)
(80, 438, 0)
(80, 442, 0)
(80, 446, 0)
(80, 450, 0)
(80, 454, 0)
(80, 458, 0)
(80, 462, 0)
(80, 466, 0)
(80, 470, 0)
(80, 474, 0)
(80, 478, 0)
(80, 482, 0)
(80, 486, 0)
(80, 490, 0)
(80, 494, 0)
(80, 498, 0)
(80, 502, 0)
(80, 506, 0)
(80, 510, 0)
(80, 514, 0)
(80, 518, 0)
(80, 522, 0)
(80, 526, 0)
(80, 530, 0)
(80, 534, 0)
(80, 538, 0)
(80, 542, 0)
(80, 546, 0)
(80, 550, 0)
(80, 554, 0)
(80, 558, 0)
(80, 562, 0)
(80, 566, 0)
(80, 570, 0)
(80, 574, 0)
(80, 578, 0)
(80, 582, 0)
(80, 586, 0)
(80, 590, 0)
(80, 594, 0)
(80, 598, 0)